### cabeçalho

#### importações

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

#### lendo as bases

In [ ]:
URL_BASES = "./../Databases/test/"
patient_csv = "patient.csv"
infusiondrug = "infusiondrug.csv"
physicalExam = "physicalExam.csv"
lab = "lab.csv"

url_patient = URL_BASES + patient_csv
url_infusiondrug = URL_BASES + infusiondrug
url_physicalExam = URL_BASES + physicalExam
url_lab = URL_BASES + lab

comeco_intervalo = 60 * 0
final_intervalo = 60 * 24

#### pacientes septicos

In [ ]:
df_patient = pd.read_csv(url_patient)
sepsis = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis', case=False, na=False)].reset_index()
sepsis = sepsis[['patientunitstayid','gender','age']]
del df_patient
sepsis.head()

### tudo sobre physical exam

#### 24 horas physical exam

In [99]:
df_physicalExam = pd.read_csv(url_physicalExam)
df_physicalExam_24 = df_physicalExam[(df_physicalExam["physicalexamoffset"]>=comeco_intervalo) &  (df_physicalExam["physicalexamoffset"]<=final_intervalo)]
df_physicalExam_24 = pd.merge(sepsis['patientunitstayid'],df_physicalExam_24,on='patientunitstayid')

del df_physicalExam

def filter_exam(df, exam_name):
    return df[df['physicalexampath'].str.contains(exam_name, case=False, na=False)].copy()


def edit_feature_exam(df,exam_name):
    df.drop(['physicalexamid', 'physicalexampath', 'physicalexamvalue'], axis=1, inplace=True)
    df.rename(columns={'physicalexamtext': exam_name}, inplace = True)

#### cada feature individual

In [100]:

ox_saturation = filter_exam(df_physicalExam_24,'O2 Sat%')
ox_saturation = filter_exam(ox_saturation,'Current')
heartRate = filter_exam(df_physicalExam_24,'HR')
heartRate =  filter_exam(heartRate,'Current')


edit_feature_exam(ox_saturation,'ox_saturation')
edit_feature_exam(heartRate,'heartRate')


heartRate = heartRate.groupby('patientunitstayid')['heartRate'].min().reset_index()
ox_saturation = ox_saturation.groupby('patientunitstayid')['ox_saturation'].min().reset_index()

/tmp/ipykernel_187083/4140761144.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return df[df['physicalexampath'].str.contains(exam_name, case=False, na=False)].copy()


#### Mean arterial presure

In [ ]:
systolic = filter_exam(df_physicalExam_24,'(systolic)')
systolic = filter_exam(systolic,'Current')
diastolic = filter_exam(df_physicalExam_24,'(diastolic)')
diastolic = filter_exam(diastolic,'Current')

edit_feature_exam(systolic,'systolic')
edit_feature_exam(diastolic,'diastolic')
'''
MAP = pd.merge(diastolic, systolic, on=['patientunitstayid','physicalexamoffset'])

# Tenta converter colunas para inteiro e usa pd.to_numeric com errors='coerce' em caso de falha
MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')
MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

# Calcula o MAP (Mean Arterial Pressure) ignorando automaticamente valores nulos
MAP['MAP'] = (MAP['systolic'] + 2 * MAP['diastolic']) / 3
'''
systolic = systolic.groupby('patientunitstayid')['systolic'].min().reset_index()

#### Glasgow Coma Scale

In [ ]:
motor = filter_exam(df_physicalExam_24,'Motor Score')
eyes = filter_exam(df_physicalExam_24,'Eyes Score')
verbal = filter_exam(df_physicalExam_24,'Verbal Score')

edit_feature_exam(motor,'motor')
edit_feature_exam(eyes,'eyes')
edit_feature_exam(verbal,'verbal')

GCS = pd.merge(motor, eyes, on=['patientunitstayid','physicalexamoffset'])
GCS = pd.merge(GCS, verbal, on=['patientunitstayid','physicalexamoffset'])
GCS['GCS'] = GCS['motor'] + GCS['verbal'] + GCS['eyes']
GCS = GCS.groupby('patientunitstayid')['GCS'].min().reset_index()

#### Merge

In [ ]:
df_physicalExam_24.to_csv((URL_BASES+'physicalexam24.csv'),index=False)

### tudo sobre lab

#### 24 horas lab

In [ ]:
df_lab = pd.read_csv(url_lab)
df_lab_24 = df_lab.loc[(df_lab["labresultoffset"] >= comeco_intervalo) & (df_lab["labresultoffset"] <= final_intervalo)]
df_lab_24 = pd.merge(sepsis['patientunitstayid'],df_lab_24,on='patientunitstayid')
df_lab_24.sort_values(by=['patientunitstayid'],inplace=True)
del df_lab

def filter_lab(df, lab_name):
    return df[df['labname'].str.contains(lab_name, case=False, na=False)].copy()

def edit_feature_lab(df,lab_name):
    df.drop(['labid','labtypeid','labname','labresultrevisedoffset'],axis=1,inplace=True)
    df.rename(columns={'labresult': lab_name, 'labresulttext' : (lab_name+'resulttext'), 'labmeasurenamesystem' : (lab_name+'.unidMedida'), 'labmeasurenameinterface': (lab_name+'.measureInterface')},inplace=True)


#### inflamatory

In [ ]:
bands = filter_lab(df_lab_24, 'bands')
CRP = filter_lab(df_lab_24, 'CRP')
ESR = filter_lab(df_lab_24, 'ESR')
Temperature = filter_lab(df_lab_24, 'Temp')
WBC = filter_lab(df_lab_24, 'WBC x 1000')

edit_feature_lab(bands,'bands')
edit_feature_lab(CRP,'CRP')
edit_feature_lab(ESR,'ESR')
edit_feature_lab(Temperature,'Temp')
Temperature['celsius'] = 0
Temperature['Tempresulttext'] = Temperature['Tempresulttext'].astype(float)
Temperature['celsius'] = Temperature['Tempresulttext'].map(lambda x: x if x < 60 else (x - 32) * 5/9)
edit_feature_lab(WBC,'WBC')

In [ ]:
CRP.drop(columns=['CRP','CRP.unidMedida','CRP.measureInterface'],inplace=True)
CRP.rename(columns={'CRPresulttext':'CRP'},inplace=True)
CRP = CRP.groupby('patientunitstayid')['CRP'].max().reset_index()
CRP.head(100)

In [ ]:
ESR.drop(columns=['ESRresulttext','ESR.unidMedida','ESR.measureInterface'],inplace=True)
ESR = ESR.groupby('patientunitstayid')['ESR'].max().reset_index()
ESR.head()

In [ ]:
WBC.drop(columns=['WBCresulttext','WBC.unidMedida','WBC.measureInterface'],inplace=True)
WBC = WBC.groupby('patientunitstayid')['WBC'].max().reset_index()
WBC.head()

In [ ]:
Temperature.drop(columns=['Temp','Tempresulttext','Temp.unidMedida','Temp.measureInterface'],inplace=True)
Temperature = Temperature.groupby('patientunitstayid')['celsius'].max().reset_index()
Temperature.head()

In [ ]:
bands['bands'] = bands['bands'].astype(float)
bands.drop(columns=['bandsresulttext','bands.unidMedida','bands.measureInterface'],inplace=True)

bands = bands.groupby('patientunitstayid')['bands'].max().reset_index()
bands.head()

In [ ]:
inflamatory = pd.merge(bands,CRP, on=['patientunitstayid'], how='outer')
inflamatory = pd.merge(inflamatory,ESR, on=['patientunitstayid'], how='outer')
inflamatory = pd.merge(inflamatory,Temperature, on=['patientunitstayid'], how='outer')
inflamatory = pd.merge(inflamatory,WBC, on=['patientunitstayid'], how='outer')
inflamatory.head()

#### Renal

In [ ]:
BUN = filter_lab(df_lab_24, 'BUN')
creatinine = filter_lab(df_lab_24, 'creatinine')

edit_feature_lab(BUN,'BUN')
edit_feature_lab(creatinine,'creatinine')

In [ ]:
BUN.drop(columns=['BUNresulttext','BUN.unidMedida','BUN.measureInterface'],inplace=True)
BUN = BUN.groupby('patientunitstayid')['BUN'].max().reset_index()
BUN.head(100)

In [ ]:
creatinine.drop(columns=['creatinineresulttext','creatinine.unidMedida','creatinine.measureInterface'],inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()
creatinine.head(1000)

In [ ]:
Renal = pd.merge(BUN,creatinine,on=['patientunitstayid'],how='outer')
Renal.head(1000)

#### cardiac

In [ ]:
bicarbonate = filter_lab(df_lab_24, 'bicarbonate')
lactate = filter_lab(df_lab_24, 'lactate')
troponin = filter_lab(df_lab_24, 'troponin')

edit_feature_lab(bicarbonate,'bicarbonate')
edit_feature_lab(lactate,'lactate')
edit_feature_lab(troponin,'troponin')

In [ ]:
bicarbonate = bicarbonate.groupby('patientunitstayid')['bicarbonate'].min().reset_index()
bicarbonate.head(1000)

In [ ]:
lactate = lactate.groupby('patientunitstayid')['lactate'].max().reset_index()
lactate.head(1000)

In [ ]:
troponin.dropna(inplace=True)
troponin = troponin.groupby('patientunitstayid')['troponin'].max().reset_index()
troponin.sort_values(by='troponinresulttext').head(10000)

In [ ]:
Cardiac = pd.merge(bicarbonate,lactate,on=['patientunitstayid'],how='outer')
Cardiac = pd.merge(Cardiac,troponin,on=['patientunitstayid'],how='outer')
Cardiac.head()

#### Pulmonary

In [ ]:
pao2 = filter_lab(df_lab_24, 'paO2')
respiratory_rate = filter_lab(df_lab_24, 'Respiratory Rate')

edit_feature_lab(pao2,'pao2')
edit_feature_lab(respiratory_rate, 'RR')

In [ ]:
pao2 = pao2.groupby('patientunitstayid')['pao2'].min().reset_index()
pao2.head(1000)

In [ ]:
respiratory_rate = respiratory_rate.groupby('patientunitstayid')['RR'].min().reset_index()
respiratory_rate.head(1000)

In [ ]:
Pulmonary = pd.merge(pao2,respiratory_rate,on=['patientunitstayid'],how='outer')
Pulmonary.head()

#### Hepatic

In [ ]:
alanine = filter_lab(df_lab_24, 'ALT')
aspartate = filter_lab(df_lab_24, 'AST')
bilirubin = filter_lab(df_lab_24, 'bilirubin')

edit_feature_lab(bilirubin,'bilirubin')
edit_feature_lab(alanine,'alanine')
edit_feature_lab(aspartate,'aspartate')

In [89]:
alanine = alanine.groupby('patientunitstayid')['alanine'].max().reset_index()
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()
aspartate = aspartate.groupby('patientunitstayid')['aspartate'].max().reset_index()

In [91]:
Hepatic = pd.merge(alanine,aspartate,on=['patientunitstayid'],how='outer')
Hepatic = pd.merge(Hepatic, bilirubin,on=['patientunitstayid'],how='outer')

Hepatic.head()

,patientunitstayid,alanine,aspartate,bilirubin
0,151179,16.0,60.0,0.2
1,156308,33.0,25.0,0.3
2,162841,1073.0,493.0,1.2
3,166175,30.0,19.0,0.8
4,190446,14.0,8.0,0.4


#### Hematologic

In [ ]:
hemoglobin = filter_lab(df_lab_24, 'Hgb')
inr = filter_lab(df_lab_24, 'INR')
platelets = filter_lab(df_lab_24, 'platelets')

edit_feature_lab(hemoglobin,'hemoglobin')
edit_feature_lab(inr,'inr')
edit_feature_lab(platelets,'platelets')

In [94]:
Hematologic = pd.merge(hemoglobin,inr,on=['patientunitstayid'],how='outer')
Hematologic = pd.merge(Hematologic,platelets,on=['patientunitstayid'],how='outer')

Hematologic.head()

,patientunitstayid,hemoglobin,inr,platelets
0,151179,12.0,NaN,541.0
1,151900,7.7,NaN,347.0
2,162841,10.4,2.1,136.0
3,166175,12.2,NaN,118.0
4,179269,9.9,NaN,135.0


#### Serum levels

In [ ]:
albumin = filter_lab(df_lab_24, 'albumin')
chloride = filter_lab(df_lab_24, 'chloride')
glucose = filter_lab(df_lab_24, 'glucose')
sodium = filter_lab(df_lab_24, 'sodium')

edit_feature_lab(albumin,'albumin')
edit_feature_lab(chloride,'chloride')
edit_feature_lab(glucose,'glucose')
edit_feature_lab(sodium,'sodium')

albumin = albumin.groupby('patientunitstayid')['albumin'].max().reset_index()
chloride = chloride.groupby('patientunitstayid')['chloride'].max().reset_index()
glucose = glucose.groupby('patientunitstayid')['glucose'].max().reset_index()
sodium = sodium.groupby('patientunitstayid')['sodium'].max().reset_index()

In [96]:
SerumLevel = pd.merge(albumin,chloride,on=['patientunitstayid'],how='outer')
SerumLevel = pd.merge(SerumLevel,glucose,on=['patientunitstayid'],how='outer')
SerumLevel = pd.merge(SerumLevel,sodium,on=['patientunitstayid'],how='outer')

SerumLevel.head()

,patientunitstayid,albumin,chloride,glucose,sodium
0,151179,1.3,115.0,109.0,148.0
1,151900,NaN,109.0,139.0,140.0
2,156308,2.3,107.0,446.0,144.0
3,162841,2.2,113.0,95.0,146.0
4,166175,2.8,99.0,322.0,132.0


#### lab final

In [ ]:
df_lab_24.head(10000)

### tudo sobre infusion drug

In [ ]:
df_infusiondrug = pd.read_csv(url_infusiondrug)
df_infusiondrug_24 = df_infusiondrug[(df_infusiondrug["infusionoffset"]>=comeco_intervalo) &  (df_infusiondrug["infusionoffset"]<=final_intervalo)]
df_infusiondrug_24 = pd.merge(sepsis['patientunitstayid'],df_infusiondrug_24,on='patientunitstayid')
del df_infusiondrug

In [ ]:
dopamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Norepinephrine', case=False, na=False)]

del df_infusiondrug_24

#### editando as features